In [1]:
import pandas as pd
import os

#### Assuming "city_hold_market_id.csv" is up to date

##### Parse mailfile

In [2]:
mail_file = pd.read_csv("campaign_e08a454a-f397-4ed6-858c-1ae848523477_mail_file_download_2022-01-05T18-07-51Z.csv", low_memory=False)

cols = list(mail_file.columns)

lower_cols = [ name.lower() for name in cols ]

mail_file.columns = lower_cols

uniq_misc = mail_file.misc.unique()

misc_lst = [ misc.split("-")[1] for misc in uniq_misc ]

##### Map mailfile market ID with holdout ID

In [3]:
hold_df = pd.read_csv("city_hold_market_id_2022-03-17.csv", low_memory=False)

hold_df = hold_df[hold_df.marketID.isin(misc_lst)]

In [41]:
hold_df

,marketID,holdoutID
3,AU,H-Austin
6,BA1,H-Gr SF 1 East Bay
10,BK,H-Brooklyn
12,BO,H-Boston
14,CB,H-Columbus
15,CH,H-Chicago
21,CT,H-Gr NYC Connecticut
25,DFW,H-Dallas
26,DN,H-Denver
32,GAU,H-Greater Austin


##### Map holdout ID with holdout filename

In [4]:
count_df = pd.read_csv("citiesHoldout_counts_2022-03-08.csv", low_memory=False)

count_df = count_df.merge(hold_df, left_on="Listcode", right_on="holdoutID", how="inner")

In [5]:
count_df

,Listcode,originalFileName,quantity,marketID,holdoutID
0,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv,26146,AU,H-Austin
1,H-Boston,StaticHoldout_2019refresh_H-Boston.forClient.csv,25829,BO,H-Boston
2,H-Brooklyn,StaticHoldout_2019refresh_H-Brooklyn.forClient...,25943,BK,H-Brooklyn
3,H-Chicago,StaticHoldout_2019refresh_H-Chicago.forClient.csv,24992,CH,H-Chicago
4,H-Columbus,StaticHoldout_2019refresh_H-Columbus.forClient...,25839,CB,H-Columbus
5,H-Dallas,StaticHoldout_2019refresh_H-Dallas.forClient.csv,24088,DFW,H-Dallas
6,H-Denver,StaticHoldout_2019refresh_H-Denver.forClient.csv,26581,DN,H-Denver
7,H-Gr Chicago,StaticHoldout_2019refresh_H-Gr Chicago.forClie...,24944,GC,H-Gr Chicago
8,H-Gr Dallas,StaticHoldout_2019refresh_H-Gr Dallas.forClien...,25005,GD,H-Gr Dallas
9,H-Gr LA LAC,StaticHoldout_2019refresh_H-Gr LA LAC.forClien...,26187,MAL,H-Gr LA LAC


##### Create holdout summary

In [2]:
def find_csv_files(path_to_dir, suffix=".csv"):
    files = os.listdir(path_to_dir)
    return [file for file in files if file.endswith(suffix)]

def file_to_df(path_to_dir):    
    file_lst = find_csv_files(path_to_dir)
    os.chdir(path_to_dir)
    dataframes = [ pd.read_csv(file, low_memory=False) for file in file_lst ]
    return dataframes, file_lst

def update_dataframes(path_to_dir):
    dataframes = file_to_df(path_to_dir)
    df_lst = dataframes[0]
    file_names = dataframes[1]

    for i, df in enumerate(df_lst):
        df['originalFileName'] = file_names[i]

    return df_lst

def final_touch(path_to_dir):
    dataframes = update_dataframes(path_to_dir)
    df_counts = [ df.groupby(['Listcode', 'originalFileName']).size().reset_index(name='quantity') for df in dataframes ]
    concated = pd.concat(df_counts)

    return concated

### Create city holdout file

In [34]:
file_lst = list(count_df.originalFileName)
file_path = "/Users/stevehan/Dropbox (Share Local Media)/Share Local Media Team Folder/Matchbacks & Misc Data/Shared Mail Files/Static Holdouts/Geographic Area Holdouts/CLIENT_READY/"

def file_to_df(path_to_dir, file_lst):    
    os.chdir(path_to_dir)
    dataframes = [ pd.read_csv(file, low_memory=False) for file in file_lst ]
    return dataframes, file_lst

def update_dataframes(path_to_dir, file_lst):
    dataframes = file_to_df(path_to_dir, file_lst)
    df_lst = dataframes[0]
    file_names = dataframes[1]

    for i, df in enumerate(df_lst):
        df['originalFileName'] = file_names[i]

    return df_lst

def create_holdout_files(path_to_dir, file_lst):
    df_lst = update_dataframes(path_to_dir, file_lst)
    df_final = pd.concat(df_lst)
    return df_final


In [35]:
create_holdout_files(file_path, file_lst)

,Full Name,Address 1,Address 2,City,St,Zip,Zip4,CRRT,Listcode,originalFileName
0,RESIDENT,1622 GARDEN ST,NaN,AUSTIN,TX,78702,5410,C001,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv
1,RESIDENT,1620 GARDEN ST,NaN,AUSTIN,TX,78702,5410,C001,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv
2,RESIDENT,1618 GARDEN ST,NaN,AUSTIN,TX,78702,5410,C001,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv
3,RESIDENT,1616 GARDEN ST,NaN,AUSTIN,TX,78702,5410,C001,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv
4,RESIDENT,1614 GARDEN ST,NaN,AUSTIN,TX,78702,5410,C001,H-Austin,StaticHoldout_2019refresh_H-Austin.forClient.csv
...,...,...,...,...,...,...,...,...,...,...
25102,RESIDENT,4848 S MILITARY TRL,NaN,LAKE WORTH,FL,33463,5311,R041,H-West Palm Beach,StaticHoldout_2021_H-West Palm Beach.forClient...
25103,RESIDENT,4552 BARCLAY CRES,NaN,LAKE WORTH,FL,33463,6023,R041,H-West Palm Beach,StaticHoldout_2021_H-West Palm Beach.forClient...
25104,RESIDENT,3819 SHERWOOD BLVD,NaN,DELRAY BEACH,FL,33445,5652,C027,H-West Palm Beach,StaticHoldout_2021_H-West Palm Beach.forClient...
25105,RESIDENT,4374 DOROTHEA DR,NaN,LAKE WORTH,FL,33463,4410,C015,H-West Palm Beach,StaticHoldout_2021_H-West Palm Beach.forClient...


##### Create holdout summary

In [36]:
def final_touch(path_to_dir, file_lst):
    dataframes = update_dataframes(path_to_dir, file_lst)
    df_counts = [ df.groupby(['Listcode', 'originalFileName']).size().reset_index(name='quantity') for df in dataframes ]
    concated = pd.concat(df_counts)
    concated = concated[["originalFileName", "Listcode", "quantity"]]

    return concated

In [39]:
final_touch(file_path, file_lst)

,originalFileName,Listcode,quantity
0,StaticHoldout_2019refresh_H-Austin.forClient.csv,H-Austin,26146
0,StaticHoldout_2019refresh_H-Boston.forClient.csv,H-Boston,25829
0,StaticHoldout_2019refresh_H-Brooklyn.forClient...,H-Brooklyn,25943
0,StaticHoldout_2019refresh_H-Chicago.forClient.csv,H-Chicago,24992
0,StaticHoldout_2019refresh_H-Columbus.forClient...,H-Columbus,25839
0,StaticHoldout_2019refresh_H-Dallas.forClient.csv,H-Dallas,24088
0,StaticHoldout_2019refresh_H-Denver.forClient.csv,H-Denver,26581
0,StaticHoldout_2019refresh_H-Gr Chicago.forClie...,H-Gr Chicago,24944
0,StaticHoldout_2019refresh_H-Gr Dallas.forClien...,H-Gr Dallas,25005
0,StaticHoldout_2019refresh_H-Gr LA LAC.forClien...,H-Gr LA LAC,26187


In [7]:
import pandas as pd
import os

# parse misc column in mail file to collect unique market IDs
def parse_mailfile(mail_file):
    mail_file_df = pd.read_csv(mail_file, low_memory=False)
    cols = list(mail_file_df.columns)
    lower_cols = [ name.lower() for name in cols ]
    mail_file_df.columns = lower_cols
    uniq_misc = mail_file_df.misc.unique()
    misc_lst = [ misc.split("-")[1] for misc in uniq_misc ]

    return misc_lst

# map unique market IDs to a holdout ID
def get_holdout_id(mail_file):
    misc_df = pd.read_csv("city_hold_market_id_2022-03-17.csv", low_memory=False)
    miscs = parse_mailfile(mail_file)
    misc_df = misc_df[misc_df.marketID.isin(miscs)]

    return misc_df

# map holdout ID to holdout file name
def get_holdout_file(mail_file):
    holdout_df = pd.read_csv("citiesHoldout_counts_2022-03-08.csv", low_memory=False)
    holdouts_to_use = get_holdout_id(mail_file)
    holdout_df = holdout_df.merge(holdouts_to_use, left_on="Listcode", right_on="holdoutID", how="inner")
    holdout_df.drop("holdoutID", axis=1, inplace=True)

    return holdout_df

# use output from get_holdout_file and get unique holdout filenames
def get_unique_file_names(mail_file):
    holdout_file_df = get_holdout_file(mail_file)
    file_lst = list(holdout_file_df.originalFileName)

    return file_lst

# pass in file list and path to static holdout to convert files to dataframes
def file_to_df(path_to_dir, mail_file):    
    file_lst = get_unique_file_names(mail_file)
    os.chdir(path_to_dir)
    dataframes = [ pd.read_csv(file, low_memory=False) for file in file_lst ]
    return dataframes, file_lst

# call file_to_df to update dataframes by creating file name column
def update_dataframes(path_to_dir, mail_file):
    dataframes = file_to_df(path_to_dir, mail_file)
    df_lst = dataframes[0]
    file_names = dataframes[1]

    for i, df in enumerate(df_lst):
        df['originalFileName'] = file_names[i]

    return df_lst

# call file_to_df to update dataframes & concat list of dataframes
def create_holdout_files(path_to_dir, mail_file):
    df_lst = update_dataframes(path_to_dir, mail_file)
    df_final = pd.concat(df_lst)
    return df_final

def final_touch(path_to_dir, mail_file):
    dataframes = update_dataframes(path_to_dir, mail_file)
    df_counts = [ df.groupby(['Listcode', 'originalFileName']).size().reset_index(name='quantity') for df in dataframes ]
    concated_df = pd.concat(df_counts)
    concated_df = concated_df[["originalFileName", "Listcode", "quantity"]]

    return concated_df

p = "/Users/stevehan/Dropbox (Share Local Media)/Mac/Documents/Notebook/StaticHoldouts/CLIENT_READY"
m = "/Users/stevehan/Dropbox (Share Local Media)/Mac/Documents/Notebook/StaticHoldouts/Test/campaign_e08a454a-f397-4ed6-858c-1ae848523477_mail_file_download_2022-01-05T18-07-51Z.csv"
m2 = "/Users/stevehan/Dropbox (Share Local Media)/Mac/Documents/Notebook/StaticHoldouts/Test/campaign_b9a62a3f-6508-4a5b-9b48-83e28d017c20_mail_file_download_2021-11-18T15-47-21Z.csv"
create_holdout_files(p, m2)

## path changes need to be reset to working

FileNotFoundError: [Errno 2] No such file or directory: 'citiesHoldout_counts_2022-03-08.csv'